In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingRegressor(
        loss=h_params[0],
        learning_rate=h_params[1],
        n_estimators=h_params[2],
        max_depth=h_params[3],
        max_features=h_params[4],
        alpha=h_params[5],
        random_state=r
    )
    # Standardize all variables (except one-hots)
    # Don't really need to do this for decision tree methods, but might as well
    # repeat the procedure that we're doing for the SVMs, just for posterity
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    ttr = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    return -np.mean(cross_val_score(
        ttr,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))

In [12]:
space = [
    # We skip "ls" since we're optimizing for mean absolute error in the h_param opt.
    Categorical(["lad", "huber", "quantile"], name="loss"),
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Real(0.5, 0.9, name="alpha"),
]
n_initial = 10 * (len(space[0].categories) + len(space[4].categories))
n_calls = 100 * (len(space[0].categories) + len(space[4].categories))

In [13]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, "opt_gbr_h_params.tsv", sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
['quantile', 0.04860832066690728, 818, 3, 'log2', 0.5039029398683332]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.0501
Function value obtained: 613.1410
Current minimum: 613.1410
Iteration No: 2 started. Evaluating function at random point.
['lad', 0.06923792863971574, 882, 3, 'log2', 0.6073125230846899]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.6389
Function value obtained: 590.8558
Current minimum: 590.8558
Iteration No: 3 started. Evaluating function at random point.
['quantile', 0.027780951796428976, 577, 3, 'log2', 0.8564436383356643]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.1983
Function value obtained: 615.0787
Current minimum: 590.8558
Iteration No: 4 started. Evaluating function at random point.
['huber', 0.04993730401749343, 291, 4, 'auto', 0.8676227203709375]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 3.9452

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 292 ended. Search finished for the next optimal point.
Time taken: 27.7995
Function value obtained: 490.0099
Current minimum: 489.6893
Iteration No: 293 started. Searching for the next optimal point.
['lad', 0.001, 770, 2, 'sqrt', 0.6910613176556772]
Iteration No: 293 ended. Search finished for the next optimal point.
Time taken: 11.5315
Function value obtained: 517.9216
Current minimum: 489.6893
Iteration No: 294 started. Searching for the next optimal point.
['huber', 0.001, 1000, 5, 'auto', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 294 ended. Search finished for the next optimal point.
Time taken: 27.7146
Function value obtained: 491.2602
Current minimum: 489.6893
Iteration No: 295 started. Searching for the next optimal point.
['lad', 0.1, 100, 2, 'log2', 0.7098101351709886]
Iteration No: 295 ended. Search finished for the next optimal point.
Time taken: 11.8129
Function value obtained: 548.0764
Current minimum: 489.6893
Iteration No: 296 started. Searching for the next optimal point.
['lad', 0.1, 100, 2, 'log2', 0.5279252771518631]
Iteration No: 296 ended. Search finished for the next optimal point.
Time taken: 11.8260
Function value obtained: 503.9135
Current minimum: 489.6893
Iteration No: 297 started. Searching for the next optimal point.
['huber', 0.001, 1000, 5, 'auto', 0.5]


...

Iteration No: 297 ended. Search finished for the next optimal point.
Time taken: 27.3350
Function value obtained: 490.6658
Current minimum: 489.6893
Iteration No: 298 started. Searching for the next optimal point.
['lad', 0.001, 854, 2, 'sqrt', 0.6914357744066628]
Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 12.1025
Function value obtained: 518.3483
Current minimum: 489.6893
Iteration No: 299 started. Searching for the next optimal point.
['lad', 0.001, 880, 2, 'sqrt', 0.6917773110368055]
Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 12.0914
Function value obtained: 516.8474
Current minimum: 489.6893
Iteration No: 300 started. Searching for the next optimal point.
['huber', 0.001, 1000, 5, 'auto', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 28.6419
Function value obtained: 490.8308
Current minimum: 489.6893
Iteration No: 301 started. Searching for the next optimal point.
['lad', 0.001, 880, 2, 'sqrt', 0.6911607241257556]
Iteration No: 301 ended. Search finished for the next optimal point.
Time taken: 11.9951
Function value obtained: 518.3921
Current minimum: 489.6893
Iteration No: 302 started. Searching for the next optimal point.
['huber', 0.001, 1000, 5, 'auto', 0.5]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 302 ended. Search finished for the next optimal point.
Time taken: 29.5739
Function value obtained: 490.8713
Current minimum: 489.6893
Iteration No: 303 started. Searching for the next optimal point.
['lad', 0.001, 905, 2, 'sqrt', 0.6908709833536575]
Iteration No: 303 ended. Search finished for the next optimal point.
Time taken: 12.6752
Function value obtained: 516.3495
Current minimum: 489.6893
Iteration No: 304 started. Searching for the next optimal point.
['lad', 0.001, 938, 2, 'sqrt', 0.6895540065162121]
Iteration No: 304 ended. Search finished for the next optimal point.
Time taken: 11.4513
Function value obtained: 517.8660
Current minimum: 489.6893
Iteration No: 305 started. Searching for the next optimal point.
['lad', 0.001, 901, 2, 'sqrt', 0.690626973024428]
Iteration No: 305 ended. Search finished for the next optimal point.
Time taken: 12.5131
Function value obtained: 517.4827
Current minimum: 489.6893
Iteration No: 306 started. Searching for the next optimal